# Instagram: Location Analysis

In [1]:
# Visual Python: Data Analysis > Import
import numpy as np
import pandas as pd
import time
import re
import os
import random

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import subprocess
import shutil
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

### Instagram 데이터 수집 함수 생성

In [3]:
#검색어 페이지 주소 생성 
def insta_search(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

In [4]:
# 첫 번재 게시글 선택
def select_first(driver):
    first = driver.find_elements(By.CSS_SELECTOR, 'div._aagu')[0]
    first.click()
    time.sleep(3)

In [5]:
# 게시글 정보 가져오기 

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    try:
        content = soup.select('div._a9zs > span')[0].text # html 태그 확인
        #content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
        
    tags = re.findall(r'#[^\s#,\\]+',content)
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    
    try:
        like = soup.select('div._aacl._aaco._aacw._aacx._aada._aade')[0].findAll('span')[-1].text # html 태그 확인
    except:
        like = 0
    
    try:
        place = soup.select('div._aaql')[0].text
        # html 태그 확인
        #place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    
    data = [content, date, like, place, tags]
    return data 

In [6]:
# 다음 게시글 조회
def move_next(driver): 
    right = driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh') # html 태그 확인
    time.sleep(1)
    right.click()
    time.sleep(1.5)

### Chrome Driver 설정

In [7]:
def chromeWebdriver():
    chrome_service = ChromeService(executable_path=ChromeDriverManager().install())
    options = Options()
    options.add_experimental_option('detach', True)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    driver = webdriver.Chrome(service=chrome_service, options=options)
    return driver

In [8]:
driver = chromeWebdriver()
#driver = webdriver.Chrome(r'C:\Users\user\chromedriver.exe')
driver.get('http://www.instagram.com')
driver.implicitly_wait(3)

In [9]:
#driver.quit()
#driver.close()

### Instagram 계정 설정

In [9]:
insta_id = '0ffthe_record'
input_id = driver.find_elements(By.CSS_SELECTOR, 'input._2hvTZ.pexuQ.zyHYP')[0] 

input_id.clear()
input_id.send_keys(insta_id)

password = 'han0725!'
input_pw = driver.find_elements(By.CSS_SELECTOR, 'input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

### Instagram 데이터 수집하기

In [12]:
%%time 

url = insta_search('southkoreatrip')
driver.get(url)
time.sleep(7)

select_first(driver)

results = [ ] 

target = 1000
for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)   
    except:
        time.sleep(4)
        move_next(driver)
    time.sleep(random.uniform(1, 4))
    if i % 100 == 0:
        print("{0}번 끝".format(i))

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']

results_df.to_csv('./data/koreagram_insta_crawling.csv', index=False, encoding='utf-8-sig')

0번 끝
100번 끝
200번 끝
300번 끝
400번 끝
500번 끝
600번 끝
700번 끝
800번 끝
900번 끝
CPU times: total: 1min 5s
Wall time: 1h 4min 46s


In [15]:
# Visual Python: Data Analysis > File
result_df = pd.read_csv('./data/southkoreatrip_insta_crawling.csv', encoding='utf-8-sig')
result_df

,content,date,like,place,tags
0,아름다운 대한민국🇰🇷 a deepening night🌒[깊어가는 밤]📍충북 음성군 ...,2022-09-03,"1,740",충북 음성,"['#충북여행', '#사찰여행', '#음성가섭사', '#힐링사진', '#충북가볼만한..."
1,..#덕수궁#서울여행#서울데이트#서울가볼만한곳#deoksugung#deoksugun...,2022-08-28,"9,621",Deoksugung,"['#덕수궁', '#서울여행', '#서울데이트', '#서울가볼만한곳', '#deok..."
2,..#창경궁#비오는날#서울여행#서울데이트#서울가볼만한곳#changgyeonggung...,2022-09-01,"9,178",창경궁 - 昌慶宮 Changgyeonggung,"['#창경궁', '#비오는날', '#서울여행', '#서울데이트', '#서울가볼만한곳..."
3,..#보보경심려#월화원#효원공원#수원가볼만한곳#수원데이트#kdramalover.Lo...,2022-08-03,"14,990",월화원,"['#보보경심려', '#월화원', '#효원공원', '#수원가볼만한곳', '#수원데이..."
4,First picture taken with @fujifilmnederland in...,2022-09-13,0,경기도송탄,"['#southkorea', '#southkoreatrip', '#amsterdam..."
...,...,...,...,...,...
994,하늘가에서 우리 ~На берегу неба я и ты,2022-08-17,108,강원도 속초,[]
995,22.08,2022-08-17,11,살곶이다리,[]
996,22.08,2022-08-17,20,서울숲,[]
997,22.08,2022-08-17,15,성수동,[]


### N개 이어서 크롤링

In [10]:
%%time

url = insta_search('koreagram')
driver.get(url)
time.sleep(20)

select_first(driver)

results = [ ] 

# 이전 n(target1)개 그냥 넘기기
target1 = 11700
for i in range(target1):
    move_next(driver)
    if i % 100 == 0:
        print("{0}번 끝".format(i))

# 뒤의 n(target2)개 이어서 크롤링
target2 = 300
for i in range(target2):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)   
    except:
        time.sleep(2)
        move_next(driver)
    time.sleep(random.uniform(1, 4))
    if i % 100 == 0:
        print("{0}번 끝".format(i))
    
print(results[:1])

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']

# 원래 파일 뒤에 이어서 저장
#results_df.to_csv('./southkoreatrip_insta_crawling.csv', index=False, mode='a', encoding='utf-8-sig', header=False)

results_df.to_csv('./koreagram_insta_crawling(0930).csv', index=False, encoding='utf-8-sig')

0번 끝
100번 끝
200번 끝
300번 끝
400번 끝
500번 끝
600번 끝
700번 끝
800번 끝
900번 끝
1000번 끝
1100번 끝
1200번 끝
1300번 끝
1400번 끝
1500번 끝
1600번 끝
1700번 끝
1800번 끝
1900번 끝
2000번 끝
2100번 끝
2200번 끝
2300번 끝
2400번 끝
2500번 끝
2600번 끝
2700번 끝
2800번 끝
2900번 끝
3000번 끝
3100번 끝
3200번 끝
3300번 끝
3400번 끝
3500번 끝
3600번 끝
3700번 끝
3800번 끝
3900번 끝
4000번 끝
4100번 끝
4200번 끝
4300번 끝
4400번 끝
4500번 끝
4600번 끝
4700번 끝
4800번 끝
4900번 끝
5000번 끝
5100번 끝


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x011ADF13+2219795]
	Ordinal0 [0x01142841+1779777]
	Ordinal0 [0x0105423D+803389]
	Ordinal0 [0x01047163+749923]
	Ordinal0 [0x01045F94+745364]
	Ordinal0 [0x01046528+746792]
	Ordinal0 [0x0104F42F+783407]
	Ordinal0 [0x0105A695+829077]
	Ordinal0 [0x0105DA60+842336]
	Ordinal0 [0x010467F6+747510]
	Ordinal0 [0x0105A4A9+828585]
	Ordinal0 [0x010AF0BA+1175738]
	Ordinal0 [0x0109E616+1107478]
	Ordinal0 [0x01077F89+950153]
	Ordinal0 [0x01078F56+954198]
	GetHandleVerifier [0x014A2CB2+3040210]
	GetHandleVerifier [0x01492BB4+2974420]
	GetHandleVerifier [0x01246A0A+565546]
	GetHandleVerifier [0x01245680+560544]
	Ordinal0 [0x01149A5C+1808988]
	Ordinal0 [0x0114E3A8+1827752]
	Ordinal0 [0x0114E495+1827989]
	Ordinal0 [0x011580A4+1867940]
	BaseThreadInitThunk [0x76E6FA29+25]
	RtlGetAppContainerNamedObjectPath [0x775F7B5E+286]
	RtlGetAppContainerNamedO

In [ ]:
result_df = pd.read_csv('./southkoreatrip_insta_crawling.csv', encoding='utf-8-sig')
result_df

### 위치정보 가져오기

In [33]:
location_counts = results_df['place'].value_counts()
print(location_counts)

                                         123
서울                                        35
금강대학교                                     26
서울 - Seoul                                15
서울숲컨서번시 Seoul Forest Park Conservancy      9
                                        ... 
고창 고인돌유적지                                  1
Cheon-An Caliente Bar 천안 깔리엔떼 바            1
Some Sevit                                 1
분당구                                        1
그레뱅 뮤지엄 Grevin Seoul Museum                1
Name: place, Length: 220, dtype: int64


In [15]:
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.to_csv('./koreagram_location_counts.csv', encoding='utf-8-sig')

In [16]:
locations = list(location_counts.index)
print(locations)

['', '서울', 'Jeonju, Korea', '대한민국', 'Seoul, South Korea', '전라남도 순천', '흰여울문화마을', '동명동', '전라북도 전주', '그라운드시소 성수', '부산', '곡성기차마을', 'Korea', 'Haeundae, Busan', '전라남도 여수', '경기도 용인', '전라남도 담양', '해미읍성', '자만벽화마을', 'Seoul Korea', '세비야', '경복궁 - Gyeongbokgung Palace', 'Songdo Beach Skywalk, Busan, South Korea', '전주 한옥마을', '올림픽공원 Oympic Park', '남대문시장 Namdaemun Global Market 韓国南大門市場', '대전광역시', '감천문화마을', '경기도 광주-Gwangju', '서울 - Seoul', 'KIAF 국제아트페어', '금강대학교', '신촌 Sinchon, Seoul, South Korea', 'Gwanghwamun', '전라남도 구례', 'Gyeongpo Lake, Gyeongpo Beach', '충청북도 청주', 'GALLERY CHOI', 'Ilsan', 'Hongdae', '광주시립미술관 Gwangju Museum of Art', '첨단', '대전', '경복궁- Gyeongbokgung Palace 景福宮', 'Myeong-dong', 'Gongju Gongsanseong Fortress Unesco World Heritage 공주 공산성 유네스코 세계문화유산', '카페너와늘', 'Namsangol Hanok Village', '클럽 크로마 Club Chroma', '시청근처', '전라남도 보성', '제주도', '이도커피_sau', 'Namsan Tower Observatory Deck', '순천카페.아삐에노', 'Busan Tower', '경주', '서울공예박물관 Seoul Museum of Craft Art', '양림미술관', '광주', 'Korean Town', '김리아갤러리', '하동 비

In [17]:
import requests 

searching = '창경궁'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)

headers = {"Authorization": "KakaoAK 7b79badcab2f1b9bd186f82a4ade78e2"}

places = requests.get(url, headers = headers).json()['documents']
print(places)

[{'address_name': '서울 종로구 와룡동 2-1', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 문화유적 > 고궁,궁', 'distance': '', 'id': '11156260', 'phone': '02-762-4868', 'place_name': '창경궁', 'place_url': 'http://place.map.kakao.com/11156260', 'road_address_name': '서울 종로구 창경궁로 185', 'x': '126.995199911733', 'y': '37.5794165384496'}, {'address_name': '서울 종로구 와룡동 2-1', 'category_group_code': 'PK6', 'category_group_name': '주차장', 'category_name': '교통,수송 > 교통시설 > 주차장', 'distance': '', 'id': '20587392', 'phone': '', 'place_name': '창경궁 주차장', 'place_url': 'http://place.map.kakao.com/20587392', 'road_address_name': '', 'x': '126.99629795858597', 'y': '37.58116450604905'}, {'address_name': '서울 종로구 와룡동 2-1', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 수목원,식물원', 'distance': '', 'id': '11002870', 'phone': '', 'place_name': '창경궁 대온실', 'place_url': 'http://place.map.kakao.com/11002870', 'road_address_name': '서울 종로구 창경궁로 185'

In [18]:
def find_places(searching):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {"Authorization": "KakaoAK 7b79badcab2f1b9bd186f82a4ade78e2"}
    
    places = requests.get(url, headers = headers).json()['documents']
    place = places[0]
    name = place['place_name']

    x = place['x']
    y = place['y']
    data = name, x, y, searching

    return data

In [19]:
print(find_places('창경궁'))

('창경궁', '126.995199911733', '37.5794165384496', '창경궁')


In [20]:
import time
from tqdm import tqdm

locations_inform = []
for location in tqdm(locations):
    try:
        data = find_places(location)
        locations_inform.append(data)
        time.sleep(0.5)
    except:
        pass

locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['네이버위치명', '경도', '위도', '인스타위치명']
locations_inform_df.to_csv('./data/koreagram_location_final.csv', index=False, encoding='utf-8-sig')

100%|████████████████████████████████████████████████████████████████████████████████| 205/205 [01:36<00:00,  2.13it/s]


### 위치 데이터 시각화 (Plotly)

In [21]:
# Visual Python: Logic > code
import plotly.express as px
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [22]:
locations_inform_df

,네이버위치명,경도,위도,인스타위치명
0,북한산둘레길 1구간소나무숲길,127.00902108350806,37.65888882945633,서울
1,전주고려병원,127.16703747285135,35.83991953035569,"Jeonju, Korea"
2,대한민국,127.062417480704,37.5067075025853,대한민국
3,순천만국가정원,127.498901621838,34.9292183256846,전라남도 순천
4,흰여울문화마을,129.04528136992104,35.077914827298564,흰여울문화마을
...,...,...,...,...
151,하이클래스153,127.77873171914088,34.69283959758517,하이클래스153
152,사직공원 전망타워,126.91152697088849,35.14163934774707,양림동
153,지중해마을,127.06164409380338,36.796950862892004,아산지중해마을
154,원조삼계탕,127.496840904758,36.1009828538327,금산원조김정이삼계탕


In [23]:
locations_inform_df.dtypes

네이버위치명    object
경도        object
위도        object
인스타위치명    object
dtype: object

In [24]:
locations_inform_df = locations_inform_df[['경도','위도']].astype('float64')

In [25]:
fig = px.scatter_mapbox(locations_inform_df, lat='위도', lon='경도', size_max=400, zoom=6)
fig.update_layout(mapbox_style = 'open-street-map')
fig.show()

In [26]:
#End of File